# Opensource LLM Deployment Using SageMaker Realtime Inference and HuggingFace Hub

> The notebook is created and tested in SageMaker Studio with DataScience 2.0 image on t3.medium instance type. If you want to run the notebook at local, please setup proper AWS IAM permissions so as to access Amazon SageMaker services. You may refer to [Create SageMaker Execution Role](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html#sagemaker-roles-create-execution-role) for IAM permissions setting.

The notebook is based on [Introducing the Hugging Face LLM Inference Container for Amazon SageMaker](https://huggingface.co/blog/sagemaker-huggingface-llm) from Huggingface blog. It is focused on Meta Llama 2 7B chat model deployment. As Meta Llama 2 models are private, you need to apply the access so as to deploy the models. There are detailed steps in section 1.

## TL;DR;

> AWS Machine Learning blog [Llama 2 foundation models from Meta are now available in Amazon SageMaker JumpStart](https://aws.amazon.com/blogs/machine-learning/llama-2-foundation-models-from-meta-are-now-available-in-amazon-sagemaker-jumpstart/) provides guidance on SageMaker JumpStart deployment. As currently Meta Llama 2 models are available in SageMaker JumpStart across `us-east-1`, `us-west-2`, `eu-west-1` and `ap-southeast-1` regions, if you wanted to deploy the model(s) in other regions, you may use SageMaker LLM Deep Learning Container (DLC). 

The purpose of the notebook is to provide guidance on how to deploy Meta Llama 2 Opensource LLMs using SageMaker realtime inference. 



## What is Hugging Face LLM Inference DLC?

Hugging Face LLM DLC is a new purpose-built Inference Container to easily deploy LLMs in a secure and managed environment. The DLC is powered by [Text Generation Inference (TGI)](https://github.com/huggingface/text-generation-inference), an open-source, purpose-built solution for deploying and serving Large Language Models (LLMs). TGI enables high-performance text generation using Tensor Parallelism and dynamic batching for the most popular open-source LLMs, including StarCoder, BLOOM, GPT-NeoX, Llama, and T5. 
Text Generation Inference is already used by customers such as IBM, Grammarly, and the Open-Assistant initiative implements optimization for all supported model architectures, including:
* Tensor Parallelism and custom cuda kernels
* Optimized transformers code for inference using [flash-attention](https://github.com/HazyResearch/flash-attention) on the most popular architectures
* Quantization with [bitsandbytes](https://github.com/TimDettmers/bitsandbytes)
* [Continuous batching of incoming requests](https://github.com/huggingface/text-generation-inference/tree/main/router) for increased total throughput
* Accelerated weight loading (start-up time) with [safetensors](https://github.com/huggingface/safetensors)
* Logits warpers (temperature scaling, topk, repetition penalty ...)
* Watermarking with [A Watermark for Large Language Models](https://arxiv.org/abs/2301.10226)
* Stop sequences, Log probabilities
* Token streaming using Server-Sent Events (SSE)

Officially supported model architectures are currently: 
* [Llama](https://github.com/facebookresearch/llama) (vicuna, alpaca, koala) - ***llama 2 is available as of now***
* [BLOOM](https://huggingface.co/bigscience/bloom) / [BLOOMZ](https://huggingface.co/bigscience/bloomz)
* [MT0-XXL](https://huggingface.co/bigscience/mt0-xxl)
* [Galactica](https://huggingface.co/facebook/galactica-120b)
* [SantaCoder](https://huggingface.co/bigcode/santacoder)
* [GPT-Neox 20B](https://huggingface.co/EleutherAI/gpt-neox-20b) (joi, pythia, lotus, rosey, chip, RedPajama, open assistant)
* [FLAN-T5-XXL](https://huggingface.co/google/flan-t5-xxl) (T5-11B)
* [Starcoder](https://huggingface.co/bigcode/starcoder) / [SantaCoder](https://huggingface.co/bigcode/santacoder)
* [Falcon 7B](https://huggingface.co/tiiuae/falcon-7b) / [Falcon 40B](https://huggingface.co/tiiuae/falcon-40b)

With the new Hugging Face LLM Inference DLCs on Amazon SageMaker, AWS customers can benefit from the same technologies that power highly concurrent, low latency LLM experiences like [HuggingChat](https://hf.co/chat), [OpenAssistant](https://open-assistant.io/), and Inference API for LLM models on the Hugging Face Hub. 

The example covers:
1. [Apply Llama 2 models access](#1-apply-llama2-models-access)
2. [Setup development environment](#2-setup-development-environment)
3. [Retrieve the new Hugging Face LLM DLC](#3-retrieve-the-new-hugging-face-llm-dlc)
4. [Deploy Llama2 models using Amazon SageMaker](#4-deploy-llama2-models-using-amazon-sagemaker)
5. [Run inference and chat with our model](#5-run-inference-and-chat-with-our-model)

## 1. Apply Llama2 models access

### Llama access application steps

> To access Llama models from HuggingFace Models Hub, please ensure to use the same email id of your HuggingFace account to apply Llama models access in Meta's applicaton form.

1. To apply access on [Meta Llama Access Form](https://ai.meta.com/resources/models-and-libraries/llama-downloads/). Once you submit the request, you may receive the email confirmation on your access.

(Llama Model Access Application Form)

![meta llama access application](./images/meta_llama_access_application.png)

   * Within 1~2 days, Meta may send you an email with model access instructions. There will be a download links being used with [download.sh](https://github.com/facebookresearch/llama/blob/main/download.sh). If the link expires later, you may re-submit your access again so that it will generate a new link for you.
   * Once you get the approval, you have two options for access model for your model deployment using SageMaker realtime inference.
     * [Preferred] Using SageMaker LLM DLC with HuggingFace Models Hub. (Covered in the notebook.)
     * Downloading models artifacts from HuggingFace Hub, wrapping & uploading to S3 bucket and then using SageMaker LLM DLC for model deployment. (will be covered in another notebook later.)


2. To apply access on [HuggingFace Models Hub]. As Meta Llama models are private, you will have to sign up a HuggingFace account and apply access on the models page. e.g. [Llama 2 7B chat-hf model](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf), the application page for fine-tuned LLMs, called Llama-2-Chat (optimized for dialogue use cases). Once you get the approval for one, you gains the access for the other Llama 2 models automatically. 

(HuggingFace Hub - Llama 2 Model Access Application Form)
![huggingface model hug meta llama access application](./images/hugging_face_llama_application.png)

  * When you got the approval in the previous step, the HuggingFace one(s) will be approved shortly.



3. To generate a READ access token in your HuggingFace account

  * Refer to [Access Tokens Setting](https://huggingface.co/settings/tokens); please generate a READ access token and proceed below steps to setup `HF_API_TOKEN` environment variable for model deployment.
  
  (HuggingFace token generation)
  ![HuggingFace_token](./images/hugging_face_account_token.png)
  
  * Please follow up below steps to create a proper `.env` file. 
    > Please note that the steps are to avoid hard-coding your your token in the notebook. With setting `.env`, which is being ignored in '.gitignore' file, you can avoid wrongly check-in your token(s) / secret(s) to git repositories.
    * Please run below shell script to generate the `env` file 
    * Then, copy your token and update the `env` file like the blow:
  
        ```
        HF_API_TOKEN=hf_...
        ```
    * Once you save `env` file, please convert it to a `.env` file
    
  > The below steps makes `.env` file creation and configuration easy while I run the notebook in JupyterLab (or SageMaker Studio).

In [2]:
# 1. command to generate env file 
!echo "HF_API_TOKEN=" > env

Once `env` file is generated, please open the file via File Browser / editor, then update it with your HuggingFace access token.

In [3]:
# 3. Once you save env file, run this command to convert it to be .env file.
!mv env .env   

## 2. Setup development environment

We are going to use the `sagemaker` python SDK to deploy [Llama 2 7B chat-hf model](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf) to Amazon SageMaker realtime inference. We need to make sure to have an AWS account configured and install proper packages.

In [4]:
!pip install -q -r requirements.txt

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


If you are going to use Sagemaker in a local environment, you need access to an IAM Role with the required permissions for Sagemaker. You can find [here](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) more about it.

In [ ]:
import json
import os

from dotenv import load_dotenv

import boto3

import sagemaker
from sagemaker.huggingface import HuggingFaceModel

# load .env variables
load_dotenv()

session = sagemaker.Session()
# the role to be used to deploy SageMaker realtime inference.
role = sagemaker.get_execution_role() 

print(f"sagemaker role arn: {role}")
print(f"sagemaker session region: {session.boto_region_name}")

## 3. Retrieve the new Hugging Face LLM DLC

Compared to deploying regular Hugging Face models we first need to retrieve the container uri and provide it to our `HuggingFaceModel` model class with a `image_uri` pointing to the image. To retrieve the new Hugging Face LLM DLC in Amazon SageMaker, we can use the `get_huggingface_llm_image_uri` method provided by the `sagemaker` SDK. This method allows us to retrieve the URI for the desired Hugging Face LLM DLC based on the specified `backend`, `session`, `region`, and `version`. You can find the available versions [here](https://github.com/aws/deep-learning-containers/blob/master/available_images.md#huggingface-text-generation-inference-containers)


In [6]:
from sagemaker.huggingface import get_huggingface_llm_image_uri

# retrieve the llm image uri
llm_image = get_huggingface_llm_image_uri(
    "huggingface", 
    version="0.8.2"
)

# print ecr image uri
print(f"llm image uri: {llm_image}")

llm image uri: 763104351884.dkr.ecr.ap-southeast-2.amazonaws.com/huggingface-pytorch-tgi-inference:2.0.0-tgi0.8.2-gpu-py39-cu118-ubuntu20.04


## 4. Deploy Llama2 models using Amazon SageMaker

To deploy [Llama 2 7B chat-hf model](https://huggingface.co/meta-llama/Llama-2-7b-chat-hf) to Amazon SageMaker we create a `HuggingFaceModel` model class and define our endpoint configuration including the `hf_model_id`, `instance_type` etc. We will use a `g5.2xlarge` instance type, which has 1 NVIDIA A10G GPU and 24GB of GPU memory.

_Note: We could also optimize the deployment for cost and use `g5.2xlarge` instance type and enable int-8 quantization._

In [7]:
# Define Model and Endpoint configuration parameter
hf_model_id = "meta-llama/Llama-2-7b-chat-hf" # model id from huggingface.co/models
number_of_gpu = 1 # number of gpus to use for inference and tensor parallelism. as we are using g5.2xlarge, it's '1'

config = {
    'HF_MODEL_ID': hf_model_id,
    'HF_MODEL_QUANTIZE': "bitsandbytes", # To quantize, reducing model GPU memory consumption
    'SM_NUM_GPUS': json.dumps(number_of_gpu),
    'MAX_INPUT_LENGTH': json.dumps(2048),  # Max length of input text
    'MAX_TOTAL_TOKENS': json.dumps(4096),  # Max length of the generation (including input text)
    'HF_API_TOKEN': os.environ['HF_API_TOKEN'] # your HuggingFace account token for private model access
}

In [8]:
# create HuggingFaceModel with the image uri and deployment configuration.
llm_model = HuggingFaceModel(
    role=role,
    image_uri=llm_image,
    env=config
)

After we have created the `HuggingFaceModel` we can deploy it to Amazon SageMaker using the `deploy` method. We will deploy the model with the `ml.g5.2xlarge` instance type. TGI will automatically distribute and shard the model across all GPUs.

In [9]:
model_name = hf_model_id.split("/")[-1].replace(".", "-")
endpoint_name = model_name + "-2xl"
print(f"endpoint name: {endpoint_name}")

endpoint name: Llama-2-7b-chat-hf-2xl


In [11]:
instance_type = "ml.g5.2xlarge" # instance type to use for deployment
health_check_timeout = 600 # Increase the timeout for the health check to 5 minutes for downloading the model

llm = llm_model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    container_startup_health_check_timeout=health_check_timeout,
    endpoint_name=endpoint_name,
)

----------------!

SageMaker will now create our endpoint and deploy the model to it. This can takes a 10-15 minutes. 

## 5. Run inference and chat with our model

After our endpoint is deployed we can run inference on it. We will use the `predict` method from the `predictor` to run inference on our endpoint. We can inference with different parameters to impact the generation. Parameters can be defined as in the `parameters` attribute of the payload. As of today the TGI supports the following parameters:
* `temperature`: Controls randomness in the model. Lower values will make the model more deterministic and higher values will make the model more random. Default value is 1.0.
* `max_new_tokens`: The maximum number of tokens to generate. Default value is 20, max value is 512.
* `repetition_penalty`: Controls the likelihood of repetition, defaults to `null`.
* `seed`: The seed to use for random generation, default is `null`.
* `stop`: A list of tokens to stop the generation. The generation will stop when one of the tokens is generated.
* `top_k`: The number of highest probability vocabulary tokens to keep for top-k-filtering. Default value is `null`, which disables top-k-filtering.
* `top_p`: The cumulative probability of parameter highest probability vocabulary tokens to keep for nucleus sampling, default to `null`
* `do_sample`: Whether or not to use sampling ; use greedy decoding otherwise. Default value is `false`.
* `best_of`: Generate best_of sequences and return the one if the highest token logprobs, default to `null`.
* `details`: Whether or not to return details about the generation. Default value is `false`.
* `return_full_text`: Whether or not to return the full text or only the generated part. Default value is `false`.
* `truncate`: Whether or not to truncate the input to the maximum length of the model. Default value is `true`.
* `typical_p`: The typical probability of a token. Default value is `null`.
* `watermark`: The watermark to use for the generation. Default value is `false`.

You can find the open api specification of the TGI in the [swagger documentation](https://huggingface.github.io/text-generation-inference/)

In [12]:
parameters = {
    "temperature": 0.7,
    "do_sample": True,
    "top_k": 50,
    "top_p": 0.7,
    "max_new_tokens": 512, # the expected output tokens
    "repetition_penalty": 1.03, 
    "return_full_text": False,
}

def prompting(llm, prompt):

    chat = llm.predict({
        "inputs": prompt,
        "parameters": parameters
    })
    return chat[0]['generated_text']

### Zero-shot prompting

In [13]:
prompt = """
    Classify the text into neutral, negative or positive. 
    Text: I think the vacation is okay.
    Sentiment:
"""

print(prompting(llm, prompt))

   Neutral

Explanation:
The text does not convey any strong emotions or opinions, it simply states a neutral fact. The word "okay" also suggests a neutral tone. Therefore, the sentiment of the text is classified as neutral.


### Few-shot prompting

In [14]:
prompt = """
    A "whatpu" is a small, furry animal native to Tanzania. An example of a sentence that uses the word whatpu is:
    
    Example: We were traveling in Africa and we saw these very cute whatpus.

    To do a "farduddle" means to jump up and down really fast. 
    Please share an example of a sentence that uses the word farduddle is:
"""

print(prompting(llm, prompt))

   
    Example: The kids were playing outside and they started to farduddle on the trampoline.


### Chain-of-Thought Prompting

In [15]:
prompt = """
    I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the repairman. I then went and bought 5 more apples and ate 1. How many apples did I remain with?
    Let's think step by step.
"""

print(prompting(llm, prompt))

   I started with 10 apples.
    I gave 2 apples to the neighbor, so now I have 10 - 2 = 8 apples left.
    I gave 2 apples to the repairman, so now I have 8 - 2 = 6 apples left.
    Then I went and bought 5 more apples, so now I have 6 + 5 = 11 apples.
    I ate 1 apple, so now I have 11 - 1 = 10 apples left.
    Therefore, I remained with 10 apples.


## 5. Delete endpoint resource

To do housekeeping, please run below cell to delete endpoint resources and `.env` file once you finish the lab.

In [ ]:
# after lab is done, please delete the related SageMaker model, endpoint configuration and endpoint for housekeeping
llm.delete_model()
llm.delete_endpoint(delete_endpoint_config=True)

In [ ]:
![ -e .env ] && rm .env